In [ ]:
import lib.facebook_scraper as fs
import pandas as pd
import numpy as np
import os
import time
pd.set_option('display.max_columns', None)

In [ ]:
fs.set_cookies('./cookies/cookies_1.txt')
# fs.enable_logging()

In [ ]:
# call back for each 10 posts
def handle_pagination_url(url):
    global resume_post_url
    resume_post_url = url
    print(f"Resume url: {url}")

# extract comments and save to global 'comment_list'
def extract_comments(comments, post_id) -> int:
    if comments is None:
        return 0
    global comment_list
    count = 0 
    for comment in comments:
        comment['post_id'] = post_id
        count += 1
        comment_list.append(comment)
        if not isinstance(comment['replies'], list):
            for reply in comment['replies']:
                reply['post_id'] = post_id
                # reply['parent_comment_id'] = comment['comment_id']
                count += 1
                comment_list.append(reply)
    return count

# extract reactors and save to global var 'reactor_list'
def extract_reactors(reactors, post_id) -> int:
    global reactor_list
    count = 0
    for reactor in reactors:
        reactor['post_id'] = post_id
        reactor_list.append(reactor)
        count += 1
    return count

# extract sharers and save to global var 'sharer_list' 
def extract_sharers(sharers, post_id) -> int:
    global sharer_list
    count = 0
    for sharer in sharers:
        sharer['post_id'] = post_id
        sharer_list.append(sharer)
        count += 1
    return count

def extract_post(post):
    num_comment = extract_comments(post['comments_full'], post['post_id'])
    print(f'\tExtracted {num_comment}/{post["comments"]} comments')

    num_reactor = extract_reactors(post['reactors'], post['post_id'])
    print(f'\tExtracted {num_reactor}/{post["reaction_count"]} reactors')

    num_sharer = extract_sharers(post['sharers'], post['post_id'])
    print(f'\tExtracted {num_sharer}/{post["shares"]} sharers')
    
    # Drop unnecessary fields: "comments_full", "reactors", "sharers" in post
    post.pop('comments_full', None)
    post.pop('reactors', None)
    post.pop('sharers', None)
    global post_list
    post_list.append(post)

In [ ]:
# Define variables
comment_list = []
reactor_list = []
sharer_list = []
post_list = []
resume_post_url = None
NUM_POSTS = 10
FANPAGE_LINK = 'vebay69'

In [ ]:
try:
    for post in fs.get_posts(
        account=FANPAGE_LINK,
        start_url=resume_post_url,
        request_url_callback=handle_pagination_url,
        options={
            "allow_extra_requests": True,
            "comments": True,
            "reactions": True,
            "comments": "generator",
            "sharers": "generator",
            "reactors": "generator",
        },
        page_limit=NUM_POSTS//10,
        timeout=300,
    ):
        print(post)
        comment = post['comments_full']
        print(f"  Start extracting post {post['post_id']}")
        extract_post(post)
        print(f"  Done {post['post_id']}")
        # t =np.random.randint(60, 360)
        t=3600
        print(f"  Sleep {t}s")
        time.sleep(t)
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

In [ ]:
comment

In [ ]:
post_list = pd.DataFrame(post_list)
post_list.head()

In [ ]:
len(sharer_list), len(comment_list), len(reactor_list), len(post_list)